In [ ]:
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
from scipy import stats
import plotly.express as px
from scripts.python.routines.plot.scatter import add_scatter_trace
import plotly.graph_objects as go
from scripts.python.routines.plot.save import save_figure
from scripts.python.routines.plot.layout import add_layout, get_axis
from statsmodels.stats.multitest import multipletests
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=False)
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
import seaborn as sns
from glob import glob
import pathlib
from sklearn.metrics import mean_absolute_error
from scipy import stats
import patchworklib as pw
import os
import functools
from scipy.stats import iqr
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
import shap
from slugify import slugify
from src.models.tabular.widedeep.ft_transformer import WDFTTransformerModel
from art.estimators.regression.pytorch import PyTorchRegressor
from art.estimators.regression.blackbox import BlackBoxRegressor
from art.attacks.evasion import ProjectedGradientDescentPyTorch, ProjectedGradientDescent
import torch
from src.tasks.metrics import get_cls_pred_metrics, get_cls_prob_metrics, get_reg_metrics


def conjunction(conditions):
    return functools.reduce(np.logical_and, conditions)


def disjunction(conditions):
    return functools.reduce(np.logical_or, conditions)

# 1. Adversarial examples for immunology data

## 1. Preparing data, model, functions for black-boxes

In [ ]:
path = "D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN/special"
path_model = f"{path}/044_small_immuno_clocks_revision/models/10_trn_val_tst/widedeep_ft_transformer_trn_val_tst/multiruns/2023-05-07_19-40-40_1337/64"
path_save = f"{path}/046_adversarial_robustness_toolbox/immuno"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
df = pd.read_excel(f"{path}/044_small_immuno_clocks_revision/figure_simage/df.xlsx", index_col=0)
feats = pd.read_excel(f"{path}/044_small_immuno_clocks_revision/feats_con_top10_new.xlsx", index_col=0).index.values
ids_feat = list(range(len(feats)))
target = 'Age'

df_preds = pd.read_excel(f"{path_model}/predictions.xlsx", index_col=0)
ids_trn = df_preds.index[df_preds['fold_0002'] == 'trn'].values
ids_val = df_preds.index[df_preds['fold_0002'] == 'val'].values
ids_tst = df_preds.index[df_preds['fold_0002'] == 'tst_ctrl_central'].values
ids_all = list(set.union(set(ids_trn), set(ids_val), set(ids_tst)))
df = df.loc[ids_all, :]

df_X = df.loc[ids_all, feats]

eps = np.array([0.4 * iqr(df.loc[ids_trn, feat].values) for feat in feats])
eps_step = np.array([0.1 * iqr(df.loc[ids_trn, feat].values) for feat in feats])

model = WDFTTransformerModel.load_from_checkpoint(checkpoint_path=f"{path_model}/best_fold_0002.ckpt")
model.eval()
model.freeze()

def predict_func_regression(X):
    model.produce_probabilities = True
    batch = {
        'all': torch.from_numpy(np.float32(X[:, ids_feat])),
        'continuous': torch.from_numpy(np.float32(X[:, ids_feat])),
        'categorical': torch.from_numpy(np.int32(X[:, []])),
    }
    tmp = model(batch)
    return tmp.cpu().detach().numpy()

art_regressor = PyTorchRegressor(
    model=model,
    loss=model.loss_fn,
    input_shape=[len(feats)],
    optimizer=torch.optim.Adam(
        params=model.parameters(),
        lr=model.hparams.optimizer_lr,
        weight_decay=model.hparams.optimizer_weight_decay
    ),
    use_amp=False,
    opt_level="O1",
    loss_scale="dynamic",
    channels_first=True,
    clip_values=None,
    preprocessing_defences=None,
    postprocessing_defences=None,
    preprocessing=(0.0, 1.0),
    device_type="cpu",
)

In [ ]:
ids_trgt = ids_tst

attack = ProjectedGradientDescent(
    estimator=art_regressor,
    norm=np.inf,
    eps=eps,
    eps_step=eps_step,
    decay=None,
    max_iter=100,
    targeted=False,
    num_random_init=0,
    batch_size=512,
    random_eps=False,
    summary_writer= False,
    verbose=True
)

X = df_X.loc[ids_trgt, :].values
X_adv = attack.generate(np.float32(df_X.loc[ids_trgt, :].values))

y_real = np.float32(df.loc[ids_trgt, target].values)
y_pred = model(torch.from_numpy(X)).cpu().detach().numpy().ravel()
y_pred_adv = model(torch.from_numpy(X_adv)).cpu().detach().numpy().ravel()

metrics = get_reg_metrics()
df_metrics = pd.DataFrame(index=[m for m in metrics], columns=['Origin', 'PGD'])
for m in metrics:
    m_val = float(metrics[m][0](torch.from_numpy(y_pred), torch.from_numpy(y_real)).numpy())
    df_metrics.at[m, 'Origin'] = m_val
    metrics[m][0].reset()
    m_val = float(metrics[m][0](torch.from_numpy(y_pred_adv), torch.from_numpy(y_real)).numpy())
    df_metrics.at[m, 'PGD'] = m_val
    metrics[m][0].reset()
df_metrics.to_excel(f"{path_save}/metrics.xlsx")